## Part 1: Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [ ]:
# Determine the number of unique values in each column
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [ ]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [ ]:
# Create a list of at least 10 column names to use as X data

columns_to_use = ['Education','Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime', 'StockOptionLevel','WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']


# Create X_df using your selected columns

X_df = attrition_df[columns_to_use]
X_df.head()


# Show the data types for X_df

X_df.dtypes


,0
Education,int64
Age,int64
DistanceFromHome,int64
JobSatisfaction,int64
OverTime,object
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64


In [ ]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)


In [ ]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

X_train['OverTime'] = X_train['OverTime'].map({'Yes': 1, 'No': 0})
X_test['OverTime'] = X_test['OverTime'].map({'Yes': 1, 'No': 0})



In [ ]:
# Create a StandardScaler

scalar = StandardScaler()


# Fit the StandardScaler to the training data

scalar.fit(X_train)


# Scale the training and testing data

X_train_scaled = scalar.transform(X_train)
X_test_scaled = scalar.transform(X_test)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
encoder = OneHotEncoder()

# Fit the encoder to the training data

encoder.fit(y_train[['Department']])


# Create two new variables by applying the encoder
# to the training and testing data

y_train_encoded_dep = encoder.transform(y_train[['Department']]).toarray()
y_test_encoded_dep = encoder.transform(y_test[['Department']]).toarray()

y_train_encoded_dep
y_test_encoded_dep



array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [ ]:
# Create a OneHotEncoder for the Attrition column

encoder = OneHotEncoder()


# Fit the encoder to the training data

encoder.fit(y_train[['Attrition']])


# Create two new variables by applying the encoder
# to the training and testing data

y_train_encoded_att = encoder.transform(y_train[['Attrition']]).toarray()
y_test_encoded_att = encoder.transform(y_test[['Attrition']]).toarray()

# Verify the shapes
print(y_train_encoded_att.shape)  # Should be (num_samples, 3) for 3 classes
print(y_test_encoded_att.shape)

# Verify that the number of unique classes matches your model output
print(len(encoder.categories_[0]))  # Number of unique classes in 'Department'


(1102, 2)
(368, 2)
2


## Part 2: Create, Compile, and Train the Model

In [ ]:
# Find the number of columns in the X training data.

number_input_features = len(X_train.columns)
print(number_input_features)

# Create the input layer

input_layer = layers.Input(shape=(number_input_features,), name = 'input')


# Create at least two shared layers

shared_layer_1 = layers.Dense(64, activation='relu', name = 'shared_layer_1')(input_layer)
shared_layer_2 = layers.Dense(128, activation='relu', name = 'shared_layer_2')(shared_layer_1)


10


In [ ]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer

department_hidden = layers.Dense(32, activation='relu',name = 'department_hidden_1')(shared_layer_2)

# Create the output layer

department_output = layers.Dense(3, activation='softmax', name = 'department_output')(department_hidden)



In [ ]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer

attrition_hidden = layers.Dense(32, activation='relu', name = 'attrition_hidden_1')(shared_layer_2)

# Create the output layer

attrition_output = layers.Dense(2, activation='sigmoid', name = 'attrition_output')(attrition_hidden)



In [ ]:
# Create the model

model = Model(inputs=input_layer, outputs=[attrition_output, department_output])


# Compile the model

model.compile(optimizer='adam', loss={'department_output' : 'categorical_crossentropy', 'attrition_output' : 'categorical_crossentropy'}, metrics={'department_output':'accuracy', 'attrition_output':'accuracy'})


# Summarize the model

model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)        │ (None, 10)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared_layer_1 (Dense)    │ (None, 64)             │            704 │ input[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared_layer_2 (Dense)    │ (None, 128)            │          8,320 │ shared_layer_1[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_hidden_1        │ (None, 32)             │          4,128 │ shared_layer_2[0][0]   │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_hidden_1       │ (None, 32)             │          4,128 │ shared_layer_2[0][0]   │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             66 │ attrition_hidden_1[0]… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             99 │ department_hidden_1[0… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
history = model.fit(X_train_scaled,
          {
              'department_output' : y_train_encoded_dep, # Now correctly targets department output
              'attrition_output' : y_train_encoded_att  # Now correctly targets attrition output
              },
          epochs=100,
          batch_size=32,
          validation_data = (X_test_scaled,
                             {

                                 'department_output' : y_test_encoded_dep, # Now correctly targets department output
                                 'attrition_output' : y_test_encoded_att,  # Now correctly targets attrition output

                                 }
                             )
          )

Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - attrition_output_accuracy: 0.6910 - attrition_output_loss: 0.5934 - department_output_accuracy: 0.4669 - department_output_loss: 1.0174 - loss: 1.6106 - val_attrition_output_accuracy: 0.8696 - val_attrition_output_loss: 0.3891 - val_department_output_accuracy: 0.6522 - val_department_output_loss: 0.8208 - val_loss: 1.1989
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - attrition_output_accuracy: 0.8274 - attrition_output_loss: 0.4281 - department_output_accuracy: 0.6406 - department_output_loss: 0.8119 - loss: 1.2398 - val_attrition_output_accuracy: 0.8668 - val_attrition_output_loss: 0.3652 - val_department_output_accuracy: 0.6522 - val_department_output_loss: 0.8118 - val_loss: 1.1678
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8507 - attrition_output_loss: 0.3761 - department_output_accuracy: 0.6279 - department_output_loss: 0.7777 - loss: 1.1542 - val_attrition_output_accuracy: 0.8696 -

In [ ]:
# Evaluate the model with the testing data

results = model.evaluate(X_test_scaled, {
    'department_output' : y_test_encoded_dep,
    'attrition_output' : y_test_encoded_att
})

results



12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - attrition_output_accuracy: 0.7953 - attrition_output_loss: 1.7426 - department_output_accuracy: 0.5094 - department_output_loss: 3.2633 - loss: 5.0167


[4.681614398956299,
 1.527323603630066,
 3.084075927734375,
 0.8179348111152649,
 0.51902174949646]

In [ ]:
# Print the accuracy for both department and attrition

print(f"Department Accuracy: {results[3]}")
print(f"Attrition Accuracy: {results[4]}")


Department Accuracy: 0.8179348111152649
Attrition Accuracy: 0.51902174949646


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. No, accuracy is not the best metric to use on this data because of the imbalance in the data. There is a significant amount of data for the 'No' attrition category, which means that the model will be biased towards predicting 'No' attrition.
2. I chose to use signmoid for the output of the attrition branch because it is a binary classification problem. I chose to use softmax for the output of the department branch because it is a multiclass classification problem.
3. One way to improve the model is to use a more balanced dataset. Another way to improve the model is to use a more complex model, such as a convolutional neural network.